데이터 정보 : (https://www.kaggle.com/datasets/cdc/national-health-and-nutrition-examination-survey)
NHANES datasets from 2013-2014

The National Health and Nutrition Examination Survey (NHANES) is a program of studies designed to assess the health and nutritional status of adults and children in the United States. The survey is unique in that it combines interviews and physical examinations. NHANES is a major program of the National Center for Health Statistics (NCHS). NCHS is part of the Centers for Disease Control and Prevention (CDC) and has the responsibility for producing vital and health statistics for the Nation.

he NHANES interview includes demographic, socioeconomic, dietary, and health-related questions. The examination component consists of medical, dental, and physiological measurements, as well as laboratory tests administered by highly trained medical personnel.

In [1]:
from google.colab import files

uploaded = files.upload()

Saving demographic.csv to demographic.csv
Saving diet.csv to diet.csv
Saving examination.csv to examination.csv
Saving labs.csv to labs.csv
Saving questionnaire.csv to questionnaire.csv


In [2]:
import pandas as pd
import numpy as np

demographic = pd.read_csv('demographic.csv', encoding='utf-8')
diet = pd.read_csv('diet.csv', encoding='utf-8')
examination = pd.read_csv('examination.csv', encoding='utf-8')
labs = pd.read_csv('labs.csv', encoding='utf-8')
questionnaire = pd.read_csv('questionnaire.csv', encoding='utf-8')

In [3]:
demographic.shape, diet.shape, examination.shape, labs.shape, questionnaire.shape

((10175, 47), (9813, 168), (9813, 224), (9813, 424), (10175, 953))

**demographic data**

SEQN : ID

RIAGENDR : gender (1: male, 2: female)  -> 'Gender'

RIDAGEYR : Age (0~79, 80 : more than 80)

RIDRETH3 : race/hispanic origin (1: maxican 2: other hispanic, 3: non-hispanic white, 4: non-hispanic black, 6: non-hispanic asian, 7: other-race: inclucing multiracial)

DMDBORN4 : country of birth

DMDEDUC3 : Education level - childern/youth 6-19 (0: x, 1~12: grade, ...)

DMDEDUC2 : education level - adults 20+ (1: less than 9, 2: 9~11, 3: highschool graduate/GED... 4: some college or AA degree, 5: college graduate or above, 7: refused, 9: DK)

DMDHHSIZ : total number of household

INDFMPIR : ratio of family income to poverty (0~4.99, 5: more than 5)

INDHHIN2 : Annual household income ( 77: refused, 99: DK)

In [4]:
demographic_extracted = demographic[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH3', 'DMDBORN4', 'DMDEDUC3', 'DMDEDUC2', 'DMDHHSIZ', 'INDFMPIR', 'INDHHIN2']]

In [ ]:
# Annual household income 'refused', 'dk' 결측치 처리 후 'Income' 변수로 변환
demographic_extracted['Income'] = demographic_extracted['INDHHIN2'].replace([77, 99], np.NaN)

In [ ]:
# 성별, 인종, 출생국 데이터를 object로 변환
demographic_extracted['Gender'] = demographic_extracted['RIAGENDR'].replace([1, 2], ['male', 'female'])
demographic_extracted['Race'] = demographic_extracted['RIDRETH3'].replace([1, 2, 3, 4, 6, 7], ['Mexican', 'Other hispanic', 'non-hispanic white', 'non-hispanic black',
                                                                                      'non-hispanic asian', 'others'])
demographic_extracted['BirthCountry'] = demographic_extracted['DMDBORN4'].replace([1, 2, 77, 99], ['US', 'Others', np.NaN, np.NaN])

demographic_extracted = demographic_extracted.drop(columns = ['RIAGENDR', 'RIDRETH3', 'DMDBORN4', 'INDHHIN2'])

In [8]:
# 아동-청소년과 성인의 교육 데이터를 통합
# 4) less than 9, 10) 9-12/9-11, 12) 13-14/high school, 14) 15/AA degree, 16) college degree

demographic_extracted['DMDEDUC3'] = demographic_extracted['DMDEDUC3'].replace([0, 1, 2, 3, 4, 5, 6, 7, 8, 55, 66, 9, 10, 11, 12, 13, 14, 15, 99],
                                                                              [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 10, 10, 10, 10, 12, 12, 14, np.NaN])
demographic_extracted['DMDEDUC2'] = demographic_extracted['DMDEDUC2'].replace([1, 2, 3, 4, 5, 7, 9], [4, 10, 12, 14, 16, np.NaN, np.NaN])

demographic_extracted['DMDEDUC3'] = demographic_extracted['DMDEDUC3'].fillna(0)
demographic_extracted['DMDEDUC2'] = demographic_extracted['DMDEDUC2'].fillna(0)

demographic_extracted['EDUC'] = demographic_extracted['DMDEDUC3'] + demographic_extracted['DMDEDUC2']
demographic_extracted['EDUC'] = demographic_extracted['EDUC'].replace(0, np.NaN)

demographic_extracted = demographic_extracted.drop(columns = ['DMDEDUC3', 'DMDEDUC2'])

In [9]:
demographic_extracted.head()

SEQN  RIDAGEYR  DMDHHSIZ  INDFMPIR  Income  Gender                Race  \
0  73557        69         3      0.84     4.0    male  non-hispanic black   
1  73558        54         4      1.78     7.0    male  non-hispanic white   
2  73559        72         2      4.51    10.0    male  non-hispanic white   
3  73560         9         4      2.52     9.0    male  non-hispanic white   
4  73561        73         2      5.00    15.0  female  non-hispanic white   

  BirthCountry  EDUC  
0           US  12.0  
1           US  12.0  
2           US  14.0  
3           US   4.0  
4           US  16.0

**labs data **

SEQN : ID

---
**백혈구**

LBXWBCSI : White blood cell count (1000 cells/uL)       # 정상범위 : 4000~10000 (4~10)

LBXLYPCT : Lymphocyte percent (%)                       # 정상범위 : 20~40%

LBXMOPCT : Monocyte percent (%)                         # 정상범위 : 10% 이하

LBXNEPCT : Segmented neutrophils percent (%)            # 정상범위 : 15~75%

LBXEOPCT : Eosinophils percent (%)                      # 정상범위 : 1~5%

LBXBAPCT : Basophils percent (%)                        # 정상범위 : 3% 이하

---
**적혈구**

LBXRBCSI : Red blood cell count (million cells/uL)      # 정상범위 : 4000000 ~ 5500000 (4~5.5)

LBXHGB ; Hemoglobin (g/dL)                              # 정상범위 : 12g 이상

LBXHCT : Hematocrit (%)                                 # 정상범위 : 35~50%

LBXMCVSI : Mean cell volume (fL)                        # 정상범위 : 80~100 fL

LBXMCHSI : Mean cell hemoglobin (pg)                    # 정상범위 : 27~33 pg

LBXMC : Mean cell hemoglobin concentration (g/dL)       # 정상범위 : 32~36 g

LBXRDW : Red cell distribution width (%)                # 정상범위 : 10~15%

---

**혈소판**

LBXPLTSI :Platelet count (1000 cells/uL)                # 정상범위 : 150000 ~ 450000 (150~450)

LBXMPSI : Mean platelet volume (fL)                     # 정상범위 : 8~12%

---

**Hepatitis A**

LBXHA : Hepatitis A antibody (1: positive, 2: negative, 3: indeterminate)

---

**Hepatitis B**

LBXHBS : Hepatitis B Surface Antibody



---

참조: https://blog.naver.com/cjd924/222531787824

In [10]:
labs_extracted = labs[['SEQN', 'LBXWBCSI', 'LBXLYPCT', 'LBXMOPCT', 'LBXNEPCT', 'LBXEOPCT', 'LBXBAPCT', 'LBXRBCSI', 'LBXHGB',
                       'LBXHCT', 'LBXMCVSI', 'LBXMCHSI', 'LBXMC', 'LBXRDW', 'LBXPLTSI', 'LBXMPSI', 'LBXHA', 'LBXHBS']]

In [ ]:
# WBC(백혈구) 정상 여부 -> 정상 : 1, 이상 : 0 

labs_extracted['LBXWBCSI'] = np.where((labs_extracted['LBXWBCSI'] >= 4) & (labs_extracted['LBXWBCSI'] <= 10), 1, 0)
labs_extracted['LBXLYPCT'] = np.where((labs_extracted['LBXLYPCT'] >= 20) & (labs_extracted['LBXLYPCT'] <= 40), 1, 0)
labs_extracted['LBXMOPCT'] = np.where(labs_extracted['LBXMOPCT'] <= 10, 1, 0)
labs_extracted['LBXNEPCT'] = np.where((labs_extracted['LBXNEPCT'] >= 15) & (labs_extracted['LBXNEPCT'] <= 75), 1, 0)
labs_extracted['LBXEOPCT'] = np.where((labs_extracted['LBXEOPCT'] >= 1) & (labs_extracted['LBXEOPCT'] <= 5), 1, 0)
labs_extracted['LBXBAPCT'] = np.where(labs_extracted['LBXBAPCT'] <= 3, 1, 0)

# WBC : 0~6. 0 이상 ~~ 6 정상
labs_extracted['WBC'] = (labs_extracted['LBXWBCSI'] + labs_extracted['LBXLYPCT'] + labs_extracted['LBXMOPCT'] +
                         labs_extracted['LBXNEPCT'] + labs_extracted['LBXEOPCT'] + labs_extracted['LBXBAPCT'])

labs_extracted = labs_extracted.drop(columns = ['LBXWBCSI', 'LBXLYPCT', 'LBXMOPCT', 'LBXNEPCT', 'LBXEOPCT', 'LBXBAPCT'])

In [12]:
# RBC(적혈구) 정상 여부 -> 정상 : 1, 이상 : 0 
labs_extracted['LBXRBCSI'] = np.where((labs_extracted['LBXRBCSI'] >= 4.5) & (labs_extracted['LBXRBCSI'] <= 5), 1, 0)
labs_extracted['LBXHGB'] = np.where(labs_extracted['LBXHGB'] >= 12, 1, 0)
labs_extracted['LBXHCT'] = np.where((labs_extracted['LBXHCT'] >= 35) & (labs_extracted['LBXHCT'] <= 50), 1, 0)
labs_extracted['LBXMCVSI'] = np.where((labs_extracted['LBXMCVSI'] >= 80) & (labs_extracted['LBXMCVSI'] <= 100), 1, 0)
labs_extracted['LBXMCHSI'] = np.where((labs_extracted['LBXMCHSI'] >= 27) & (labs_extracted['LBXMCHSI'] <= 33), 1, 0)
labs_extracted['LBXMC'] = np.where((labs_extracted['LBXMC'] >= 32) & (labs_extracted['LBXMC'] <= 36), 1, 0)
labs_extracted['LBXRDW'] = np.where((labs_extracted['LBXRDW'] >= 10) & (labs_extracted['LBXRDW'] <= 15), 1, 0)

# RBC : 0~7. 0 이상 ~~ 7 정상
labs_extracted['RBC'] = (labs_extracted['LBXRBCSI'] + labs_extracted['LBXHGB'] + labs_extracted['LBXHCT'] +
                         labs_extracted['LBXMCVSI'] + labs_extracted['LBXMCHSI'] + labs_extracted['LBXMC'] + labs_extracted['LBXRDW'])

labs_extracted = labs_extracted.drop(columns = ['LBXRBCSI', 'LBXHGB', 'LBXHCT', 'LBXMCVSI', 'LBXMCHSI', 'LBXMC', 'LBXRDW'])

In [13]:
# PLT(혈소판) 정상 여부 -> 정상 : 1, 이상: 0

labs_extracted['LBXPLTSI'] = np.where((labs_extracted['LBXPLTSI'] >= 150) & (labs_extracted['LBXPLTSI'] <= 450), 1, 0)
labs_extracted['LBXMPSI'] = np.where((labs_extracted['LBXMPSI'] >= 8) & (labs_extracted['LBXMPSI'] <= 12), 1, 0)

# PLT : 0~2. 0 이상 ~~ 2 정상
labs_extracted['PLT'] = labs_extracted['LBXPLTSI'] + labs_extracted['LBXMPSI']

labs_extracted = labs_extracted.drop(columns = ['LBXPLTSI', 'LBXMPSI'])

In [14]:
labs_extracted.head()

SEQN  LBXHA  LBXHBS  WBC  RBC  PLT
0  73557    1.0     1.0    4    6    2
1  73558    2.0     2.0    4    5    2
2  73559    2.0     2.0    3    6    2
3  73560    1.0     2.0    6    7    2
4  73561    2.0     2.0    6    7    2

**Dietary Interview - Individual Foods, First Day**

WTDRD1 : Dietary day one sample weight

WTDR2D : Dietary two-day sample weight

DR1IKCAL : Energy (kcal)

DR1IPROT : Protein (gm)

DR1ICARB : Carbohydrate (gm)

DR1ISUGR : Total sugars (gm)

DR1IFIBE : Dietary fiber (gm)

DR1ITFAT : Total fat (gm)

DR1ISFAT : Total saturated fatty acids (gm)

DR1IMFAT : Total monounsaturated fatty acids (gm)

DR1IPFAT : Total polyunsaturated fatty acids (gm)

DR1ICHOL : Cholesterol (mg)

DR1ICAFF : Caffeine (mg)

DR1ITHEO : Theobromine (mg)

DR1IALCO : Alcohol (gm)

DR1IMOIS : Moisture (gm)


---

micronutritients


(Anaemia : iron, folate, vitamins B12, vitamin A

brain damage + spontaneous abortion and congenital anomalies, stillbirth during pregnance : iodine

preventable blindness in children and increases the risk of disease and death from severe infections such as diarrhoeal disease and measles : Vitamin A)


---


iron : DR1IIRON : Iron (mg)

folate : DR1TFOLA : Total folate (mcg)

vitamins B12 : DR1IVB12 : Vitamin B12 (mcg)

vitamin A : DR1IVARA : Vitamin A as retinol activity equivalents (mcg)


In [15]:
diet_extracted = diet[['SEQN', 'WTDRD1', 'WTDR2D', 'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR', 'DR1TFIBE', 'DR1TTFAT', 
                       'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL', 'DR1TCAFF', 'DR1TTHEO', 'DR1TALCO', 'DR1TMOIS', 'DR1TIRON', 'DR1TFOLA', 'DR1TVARA', 'DR1TVB12']]

In [ ]:
# 첫번째 날과 두번째 날에 먹은 음식의 평균 피처 생성 : 'EAT'
# 두 번째 날의 결측치는 첫 째날 식사량의 평균으로 지정
WTDRD1_mean = diet_extracted['WTDRD1'].mean()
diet_extracted['WTDR2D'] = diet_extracted['WTDR2D'].replace(0, WTDRD1_mean)

diet_extracted['EAT'] = (diet_extracted['WTDRD1'] + diet_extracted['WTDR2D']) / 2
diet_extracted = diet_extracted.drop(columns=['WTDRD1', 'WTDR2D'])

**Examination Data**

Body Measures	

BMXWT : Weight (kg)

BMXHT : Standing Height (cm)

BMXBMI : Body Mass Index (kg/m**2)

---

Muscle Strength - Grip Test

MGDCGSZ : Combined grip strength (kg): the sum of the largest reading from each hand.


In [17]:
examination_extracted = examination[['SEQN', 'BMXWT', 'BMXHT', 'BMXBMI', 'MGDCGSZ']]

**Questionnair Data**

**Consumer Behavior**	
CBD070 : The next questions are about how much money {your family spends/you spend} on food. First I'll ask you about money spent at supermarkets or grocery stores. Then we will talk about money spent at other types of stores. During the past 30 days, how much money {did your family/did you} spend at supermarkets or grocery stores? Please include purchases made with food stamps. ( 7777: refused, 99999 : DK)

CBD110 : About how much money {did your family/did you} spend on food at these types of stores? (Please do not include any stores you have already told me about.) ( 7777: refused, 99999 : DK)

CBD120 : CODE AS DOLLAR AMOUNT OF MONEY SPENT IN 30 DAYS. IF RESPONDENT KNOWS ONLY AMOUNT FOR SELF, CODE DK. ENTER "0" IF SP SAYS NO MONEY WAS SPENT.

CBD130 : CODE AS DOLLAR AMOUNT OF MONEY SPENT IN 30 DAYS. IF RESPONDENT KNOWS ONLY AMOUNT FOR SELF, CODE DK. ENTER "0" IF SP SAYS NO MONEY WAS SPENT.

---

**Hospital Utilization & Access to Care	**

HUQ010 : {First/Next} I have some general questions about {your/SP's} health. Would you say {your/SP's} health in general is . . . (1: Excellent ... 5: poor, 7: refused, 9:dk)

HUQ041 : {What kind of place is it - a clinic, doctor's office, emergency room, or some other place?} {What kind of place {do you/does SP} go to most often - a clinic, doctor's office, emergency room, or some other place?}

HUQ051 : {During the past 12 months, how/How} many times {have you/has SP} seen a doctor or other health care professional about {your/his/her} health at a doctor's office, a clinic or some other place? Do not include times {you were/s/he was} hospitalized overnight, visits to hospital emergency rooms, home visits or telephone calls.

---

**Housing Characteristics**	

HOD050 : How many rooms are in this home? Count the kitchen but not the bathroom.

HOQ065 : Is this {mobile home/house/apartment} owned, being bought, rented, or occupied by some other arrangement by {you/you or someone else in your family}?

In [18]:
questionnaire_extracted = questionnaire[['SEQN', 'CBD070', 'CBD110', 'CBD120', 'CBD130', 'HUQ010', 'HUQ041', 'HUQ051', 'HOD050', 'HOQ065']]

In [ ]:
# refused/Don't know 결측치 처리 => 평균치
questionnaire_extracted.iloc[:, 1:6] = questionnaire_extracted.iloc[:, 1:6].replace([777777, 999999], np.NaN)
questionnaire_extracted['CBD070'] = questionnaire_extracted['CBD070'].fillna(questionnaire_extracted['CBD070'].mean())
questionnaire_extracted['CBD110'] = questionnaire_extracted['CBD110'].fillna(questionnaire_extracted['CBD110'].mean())
questionnaire_extracted['CBD120'] = questionnaire_extracted['CBD120'].fillna(questionnaire_extracted['CBD120'].mean())
questionnaire_extracted['CBD130'] = questionnaire_extracted['CBD130'].fillna(questionnaire_extracted['CBD130'].mean())

# 식비 feature 생성 : SpendingFood
questionnaire_extracted['SpendingFood'] = questionnaire_extracted['CBD070'] + questionnaire_extracted['CBD110'] + questionnaire_extracted['CBD120'] + questionnaire_extracted['CBD130']
questionnaire_extracted = questionnaire_extracted.drop(columns = ['CBD070', 'CBD110', 'CBD120', 'CBD130'])

In [20]:
# 건강 평가 지표 : refused/don't know 결측치 처리 + 건강에 우호적인 평가를 더 큰 수로 변환 (5~1) -> (1~5)
questionnaire_extracted['HUQ010'] = questionnaire_extracted['HUQ010'].replace([7, 9], np.NaN)
questionnaire_extracted['HUQ010'] = questionnaire_extracted['HUQ010'].replace([1, 2, 3, 4, 5], [5, 4, 3, 2, 1])

In [21]:
# 건강을 위해 방문하는 시설 피처를 object로 변환, '가지 않는다' 처리 및 결측치 지정
questionnaire_extracted['HUQ041'] = questionnaire_extracted['HUQ041'].fillna("Don't Go")
questionnaire_extracted['HUQ041'] = questionnaire_extracted['HUQ041'].replace(77, np.NaN)
questionnaire_extracted['HUQ041'] = questionnaire_extracted['HUQ041'].replace([1, 2, 3, 4, 5, 6], ['Clinic or health center', "Doctor's office or HMO", 
                                                                                                   'Hospital emergency room', 'Hospital outpatient department', 'Etc', 'Etc'])

# 얼마나 자주 가는지에 대한 피쳐를 방문 횟수에 맞게 변환
questionnaire_extracted['HUQ051'] = questionnaire_extracted['HUQ051'].replace([2, 3, 4, 5, 6, 7, 8, 99], [2.5, 4.5, 6.5, 8.5, 11, 14, 16, np.NaN])

In [22]:
# 방 갯수 결측치 처리
questionnaire_extracted['HOD050'] = questionnaire_extracted['HOD050'].replace([777, 999], np.NaN)

# 주거 형태를 object로 변환
questionnaire_extracted['HOQ065'] = questionnaire_extracted['HOQ065'].replace([1, 2, 3, 7, 9], ['Owned or being bought', 'Rented', 'Other arrangement', np.NaN, np.NaN])

In [23]:
questionnaire_extracted.head()

SEQN  HUQ010                  HUQ041  HUQ051  HOD050  \
0  73557     4.0  Doctor's office or HMO     8.5     4.0   
1  73558     2.0  Doctor's office or HMO     8.5     7.0   
2  73559     3.0  Doctor's office or HMO     2.5     6.0   
3  73560     4.0                Don't Go     1.0     5.0   
4  73561     1.0  Doctor's office or HMO     6.5     8.0   

                  HOQ065  SpendingFood  
0                 Rented         435.0  
1  Owned or being bought         830.0  
2  Owned or being bought         190.0  
3  Owned or being bought         530.0  
4  Owned or being bought         240.0

In [24]:
# ID number인 'SEQN'을 중심으로 데이터 병합

df = pd.merge(demographic_extracted, questionnaire_extracted, on='SEQN', how='outer')
df_2 = pd.merge(labs_extracted, diet_extracted, how='outer', on='SEQN')
df_2 = pd.merge(df_2, examination_extracted, how='outer', on='SEQN')
df = pd.merge(df, df_2, how='outer', on='SEQN')

In [25]:
# 6세 이상만 추출
df = df[df['RIDAGEYR'] > 5]

In [26]:
df.shape

(8572, 43)

iron 적절량: 
남 청소년 11 ~ 45g, 여 청소년 15 ~ 45g, 남 성인 10 ~ 45g, 여 성인 14 ~ 45g, 노년층 9-45g  (https://health.chosun.com/site/data/html_dir/2020/02/03/2020020303924.html)

In [27]:
df = df.reset_index(drop=True)
df.head()

SEQN  RIDAGEYR  DMDHHSIZ  INDFMPIR  Income  Gender                Race  \
0  73557        69         3      0.84     4.0    male  non-hispanic black   
1  73558        54         4      1.78     7.0    male  non-hispanic white   
2  73559        72         2      4.51    10.0    male  non-hispanic white   
3  73560         9         4      2.52     9.0    male  non-hispanic white   
4  73561        73         2      5.00    15.0  female  non-hispanic white   

  BirthCountry  EDUC  HUQ010  ... DR1TMOIS  DR1TIRON  DR1TFOLA DR1TVARA  \
0           US  12.0     4.0  ...  2701.79      8.41     285.0    110.0   
1           US  12.0     2.0  ...  6779.99     26.88    1243.0   1488.0   
2           US  14.0     3.0  ...  3766.70     17.57     423.0    655.0   
3           US   4.0     4.0  ...   980.45     14.19     275.0    545.0   
4           US  16.0     1.0  ...  1029.69     17.72     390.0    863.0   

   DR1TVB12            EAT  BMXWT  BMXHT  BMXBMI  MGDCGSZ  
0      2.79   14909.609256   78.3  171.3    26.7     55.2  
1     21.45   15308.146367   89.5  176.8    28.6     61.5  
2      3.78   49518.024820   88.9  175.3    28.9     91.0  
3      8.76  134084.718179   32.2  137.3    17.1     32.2  
4      8.30   49028.625013   52.0  162.4    19.7     30.9  

[5 rows x 43 columns]

In [ ]:
# iron 기준 :  RIDAGEYR, Gender, DR1TIRON

df['IRON'] = df['DR1TIRON'] 

for i in range(len(df)):
    if df['RIDAGEYR'][i] >= 60:
      if 45 >= df['IRON'][i] >= 9:
        df['IRON'][i] = 1
      else:
        df['IRON'][i] = 0
    elif 60 > df['RIDAGEYR'][i] >= 19:
      if df['Gender'][i] == 'female':
        if 45 >= df['IRON'][i] >= 14:
          df['IRON'][i] = 1
        else:
          df['IRON'][i] = 0
      else:
        if 45 >= df['IRON'][i] >= 10:
          df['IRON'][i] = 1
        else:
          df['IRON'][i] = 0
    else:
      if df['Gender'][i] == 'female':
        if 45 >= df['IRON'][i] >= 15:
          df['IRON'][i] = 1
        else:
          df['IRON'][i] = 0
      else:
        if 45 >= df['IRON'][i] >= 11:
          df['IRON'][i] = 1
        else:
          df['IRON'][i] = 0

In [29]:
df['IRON'].value_counts()

0.0    4493
1.0    4079
Name: IRON, dtype: int64

folate 적절량 :


6 ~ 8세 180 ~ 500,// 9 ~ 11세 250 ~ 600,// 12 ~ 14세 300 ~ 800,// 15 ~ 18세 320~900, // 19 ~ 세 320 ~ 1000

https://terms.naver.com/entry.naver?docId=6028730&cid=67241&categoryId=67241

In [ ]:
# DR1TFOLA , RIDAGEYR, Gender

df['FOLATE'] = df['DR1TFOLA']

for i in range(len(df)):
    if df['RIDAGEYR'][i] >= 19:
      if 1000 >= df['FOLATE'][i] >= 320:
        df['FOLATE'][i] = 1
      else:
        df['FOLATE'][i] = 0
    elif 19 > df['RIDAGEYR'][i] >= 15:
      if 900 >= df['FOLATE'][i] >= 320:
        df['FOLATE'][i] = 1
      else:
        df['FOLATE'][i] = 0
    elif 15 > df['RIDAGEYR'][i] >= 12:
      if 800 >= df['FOLATE'][i] >= 300:
        df['FOLATE'][i] = 1
      else:
        df['FOLATE'][i] = 0
    elif 12 > df['RIDAGEYR'][i] >= 9:
      if 600 >= df['FOLATE'][i] >= 250:
        df['FOLATE'][i] = 1
      else: 
        df['FOLATE'][i] = 0
    else:
      if 500 >= df['FOLATE'][i] >= 180:
        df['FOLATE'][i] = 1
      else:
        df['FOLATE'][i] = 0

In [31]:
df['FOLATE'].value_counts()

0.0    4695
1.0    3877
Name: FOLATE, dtype: int64

Vitamin B12:

~ 8세 1.2mcg 이상, 9 ~ 13세 1.8mcg 이상, 14세 이상 2.4mcg 이상

https://baroblog.tistory.com/522

In [32]:
# DR1IVB12, RIDAGEYR

df['VitB12'] = df['DR1TVB12']

for i in range(len(df)):
    if df['RIDAGEYR'][i] >= 14:
      if df['VitB12'][i] >= 2.4:
        df['VitB12'][i] = 1
      else:
        df['VitB12'][i] = 0
    elif 14 > df['RIDAGEYR'][i] >= 9:
      if df['VitB12'][i] >= 1.8:
        df['VitB12'][i] = 1
      else:
        df['VitB12'][i] = 0
    else:
      if df['VitB12'][i] >= 1.2:
        df['VitB12'][i] = 1
      else:
        df['VitB12'][i] = 0

In [33]:
df['VitB12'].value_counts()

1.0    5301
0.0    3271
Name: VitB12, dtype: int64

Vitamin A

6-8세, 9-11세, 12-14세, 15-18세, 19-29세, 30-49세, 50-64세, 65 이상

남: 320 - 1000, 420 - 1500, 540 - 2100, 620 - 2300, 570 - 3000, 550 - 3000, 530 - 3000, 500 - 3000

여: 290 - 1000, 380 - 1500, 470 - 2100, 440 - 2300, 460 - 3000, 450 - 3000, 430 - 3000, 410 - 3000

https://terms.naver.com/entry.naver?docId=6028749&cid=67241&categoryId=67241

In [ ]:
# DR1TVARA, RIDAGEYR, Gender

df['VitA'] = df['DR1TVARA']

for i in range(len(df)):
    if df['Gender'][i] == 'male':
      if df['RIDAGEYR'][i] >= 65:
        if 3000 >= df['VitA'][i] >= 500:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 65 > df['RIDAGEYR'][i] >= 50:
        if 3000 >= df['VitA'][i] >= 530:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 50 > df['RIDAGEYR'][i] >= 30:
        if 3000 >= df['VitA'][i] >= 550:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 30 > df['RIDAGEYR'][i] >= 19:
        if 3000 >= df['VitA'][i] >= 570:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 19 > df['RIDAGEYR'][i] >= 15:
        if 2300 >= df['VitA'][i] >= 620:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 15 > df['RIDAGEYR'][i] >= 12:
        if 2100 >= df['VitA'][i] >= 540:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 12 > df['RIDAGEYR'][i] >= 9:
        if 1500 >= df['VitA'][i] >= 420:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      else:
        if 1000 >= df['VitA'][i] >= 320:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0

    else:
      if df['RIDAGEYR'][i] >= 65:
        if 3000 >= df['VitA'][i] >= 410:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 65 > df['RIDAGEYR'][i] >= 50:
        if 3000 >= df['VitA'][i] >= 430:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 50 > df['RIDAGEYR'][i] >= 30:
        if 3000 >= df['VitA'][i] >= 450:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 30 > df['RIDAGEYR'][i] >= 19:
        if 3000 >= df['VitA'][i] >= 460:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 19 > df['RIDAGEYR'][i] >= 15:
        if 2300 >= df['VitA'][i] >= 440:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 15 > df['RIDAGEYR'][i] >= 12:
        if 2100 >= df['VitA'][i] >= 470:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      elif 12 > df['RIDAGEYR'][i] >= 9:
        if 1500 >= df['VitA'][i] >= 380:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0
      else:
        if 1000 >= df['VitA'][i] >= 290:
          df['VitA'][i] = 1
        else:
          df['VitA'][i] = 0

In [35]:
df['VitA'].value_counts()

0.0    4977
1.0    3595
Name: VitA, dtype: int64

In [36]:
df.head()

SEQN  RIDAGEYR  DMDHHSIZ  INDFMPIR  Income  Gender                Race  \
0  73557        69         3      0.84     4.0    male  non-hispanic black   
1  73558        54         4      1.78     7.0    male  non-hispanic white   
2  73559        72         2      4.51    10.0    male  non-hispanic white   
3  73560         9         4      2.52     9.0    male  non-hispanic white   
4  73561        73         2      5.00    15.0  female  non-hispanic white   

  BirthCountry  EDUC  HUQ010  ... DR1TVB12            EAT  BMXWT  BMXHT  \
0           US  12.0     4.0  ...     2.79   14909.609256   78.3  171.3   
1           US  12.0     2.0  ...    21.45   15308.146367   89.5  176.8   
2           US  14.0     3.0  ...     3.78   49518.024820   88.9  175.3   
3           US   4.0     4.0  ...     8.76  134084.718179   32.2  137.3   
4           US  16.0     1.0  ...     8.30   49028.625013   52.0  162.4   

   BMXBMI  MGDCGSZ  IRON  FOLATE  VitB12  VitA  
0    26.7     55.2   0.0     0.0     1.0   0.0  
1    28.6     61.5   1.0     0.0     1.0   1.0  
2    28.9     91.0   1.0     1.0     1.0   1.0  
3    17.1     32.2   1.0     1.0     1.0   1.0  
4    19.7     30.9   1.0     1.0     1.0   1.0  

[5 rows x 47 columns]

In [37]:
df = df.rename(columns = {'RIDAGEYR': 'Age', 'DMDHHSIZ': 'Household_number', 'INDFMPIR': 'Income ratio', 'LBXHA': 'Hepatitis A', 'LBXHBS': 'Hepatitis B'})
df.head()

SEQN  Age  Household_number  Income ratio  Income  Gender  \
0  73557   69                 3          0.84     4.0    male   
1  73558   54                 4          1.78     7.0    male   
2  73559   72                 2          4.51    10.0    male   
3  73560    9                 4          2.52     9.0    male   
4  73561   73                 2          5.00    15.0  female   

                 Race BirthCountry  EDUC  HUQ010  ... DR1TVB12            EAT  \
0  non-hispanic black           US  12.0     4.0  ...     2.79   14909.609256   
1  non-hispanic white           US  12.0     2.0  ...    21.45   15308.146367   
2  non-hispanic white           US  14.0     3.0  ...     3.78   49518.024820   
3  non-hispanic white           US   4.0     4.0  ...     8.76  134084.718179   
4  non-hispanic white           US  16.0     1.0  ...     8.30   49028.625013   

   BMXWT  BMXHT  BMXBMI  MGDCGSZ  IRON  FOLATE  VitB12  VitA  
0   78.3  171.3    26.7     55.2   0.0     0.0     1.0   0.0  
1   89.5  176.8    28.6     61.5   1.0     0.0     1.0   1.0  
2   88.9  175.3    28.9     91.0   1.0     1.0     1.0   1.0  
3   32.2  137.3    17.1     32.2   1.0     1.0     1.0   1.0  
4   52.0  162.4    19.7     30.9   1.0     1.0     1.0   1.0  

[5 rows x 47 columns]

In [38]:
df = df.rename(columns = {'HUQ010': 'Subjective health', 'HUQ041': 'Health place', 'HUQ051': 'Visiting number', 'HOD050': 'Room', 'HOQ065': 'Housing type'})
df.head()

SEQN  Age  Household_number  Income ratio  Income  Gender  \
0  73557   69                 3          0.84     4.0    male   
1  73558   54                 4          1.78     7.0    male   
2  73559   72                 2          4.51    10.0    male   
3  73560    9                 4          2.52     9.0    male   
4  73561   73                 2          5.00    15.0  female   

                 Race BirthCountry  EDUC  Subjective health  ... DR1TVB12  \
0  non-hispanic black           US  12.0                4.0  ...     2.79   
1  non-hispanic white           US  12.0                2.0  ...    21.45   
2  non-hispanic white           US  14.0                3.0  ...     3.78   
3  non-hispanic white           US   4.0                4.0  ...     8.76   
4  non-hispanic white           US  16.0                1.0  ...     8.30   

             EAT  BMXWT  BMXHT  BMXBMI  MGDCGSZ  IRON  FOLATE  VitB12  VitA  
0   14909.609256   78.3  171.3    26.7     55.2   0.0     0.0     1.0   0.0  
1   15308.146367   89.5  176.8    28.6     61.5   1.0     0.0     1.0   1.0  
2   49518.024820   88.9  175.3    28.9     91.0   1.0     1.0     1.0   1.0  
3  134084.718179   32.2  137.3    17.1     32.2   1.0     1.0     1.0   1.0  
4   49028.625013   52.0  162.4    19.7     30.9   1.0     1.0     1.0   1.0  

[5 rows x 47 columns]

In [39]:
df = df.rename(columns = {'DR1TKCAL': 'Energy(kcal)', 'DR1TPROT': 'Protein', 'DR1TCARB': 'Carbohydrate', 'DR1TSUGR': 'Sugars', 'DR1TFIBE': 'Fiber',
                          'DR1TTFAT': 'Fat', 'DR1TSFAT': 'Saturated Fatty Acids', 'DR1TMFAT': 'monounsaturated fatty acids', 'DR1TPFAT': 'Polyunsaturated Fatty Acids',
                          'DR1TCHOL': 'Cholesterol', 'DR1TCAFF': 'Caffeine', 'DR1TTHEO': 'Theobromine', 'DR1TALCO': 'Alcohol', 'DR1TMOIS' : 'Moisture'})
df.head()

SEQN  Age  Household_number  Income ratio  Income  Gender  \
0  73557   69                 3          0.84     4.0    male   
1  73558   54                 4          1.78     7.0    male   
2  73559   72                 2          4.51    10.0    male   
3  73560    9                 4          2.52     9.0    male   
4  73561   73                 2          5.00    15.0  female   

                 Race BirthCountry  EDUC  Subjective health  ... DR1TVB12  \
0  non-hispanic black           US  12.0                4.0  ...     2.79   
1  non-hispanic white           US  12.0                2.0  ...    21.45   
2  non-hispanic white           US  14.0                3.0  ...     3.78   
3  non-hispanic white           US   4.0                4.0  ...     8.76   
4  non-hispanic white           US  16.0                1.0  ...     8.30   

             EAT  BMXWT  BMXHT  BMXBMI  MGDCGSZ  IRON  FOLATE  VitB12  VitA  
0   14909.609256   78.3  171.3    26.7     55.2   0.0     0.0     1.0   0.0  
1   15308.146367   89.5  176.8    28.6     61.5   1.0     0.0     1.0   1.0  
2   49518.024820   88.9  175.3    28.9     91.0   1.0     1.0     1.0   1.0  
3  134084.718179   32.2  137.3    17.1     32.2   1.0     1.0     1.0   1.0  
4   49028.625013   52.0  162.4    19.7     30.9   1.0     1.0     1.0   1.0  

[5 rows x 47 columns]

In [40]:
df.columns.tolist()

['SEQN',
 'Age',
 'Household_number',
 'Income ratio',
 'Income',
 'Gender',
 'Race',
 'BirthCountry',
 'EDUC',
 'Subjective health',
 'Health place',
 'Visiting number',
 'Room',
 'Housing type',
 'SpendingFood',
 'Hepatitis A',
 'Hepatitis B',
 'WBC',
 'RBC',
 'PLT',
 'Energy(kcal)',
 'Protein',
 'Carbohydrate',
 'Sugars',
 'Fiber',
 'Fat',
 'Saturated Fatty Acids',
 'monounsaturated fatty acids',
 'Polyunsaturated Fatty Acids',
 'Cholesterol',
 'Caffeine',
 'Theobromine',
 'Alcohol',
 'Moisture',
 'DR1TIRON',
 'DR1TFOLA',
 'DR1TVARA',
 'DR1TVB12',
 'EAT',
 'BMXWT',
 'BMXHT',
 'BMXBMI',
 'MGDCGSZ',
 'IRON',
 'FOLATE',
 'VitB12',
 'VitA']

In [41]:
df = df.drop(columns= ['DR1TIRON', 'DR1TFOLA', 'DR1TVARA', 'DR1TVB12'])

In [42]:
df = df.drop(columns = 'BMXWT')

In [43]:
df = df.rename(columns = {'BMXHT': 'Height', 'BMXBMI': 'BMI', 'MGDCGSZ': 'Grip Strength'})
df.head()

SEQN  Age  Household_number  Income ratio  Income  Gender  \
0  73557   69                 3          0.84     4.0    male   
1  73558   54                 4          1.78     7.0    male   
2  73559   72                 2          4.51    10.0    male   
3  73560    9                 4          2.52     9.0    male   
4  73561   73                 2          5.00    15.0  female   

                 Race BirthCountry  EDUC  Subjective health  ... Alcohol  \
0  non-hispanic black           US  12.0                4.0  ...     0.0   
1  non-hispanic white           US  12.0                2.0  ...   119.0   
2  non-hispanic white           US  14.0                3.0  ...     0.0   
3  non-hispanic white           US   4.0                4.0  ...     0.0   
4  non-hispanic white           US  16.0                1.0  ...     0.0   

   Moisture            EAT Height   BMI  Grip Strength  IRON  FOLATE  VitB12  \
0   2701.79   14909.609256  171.3  26.7           55.2   0.0     0.0     1.0   
1   6779.99   15308.146367  176.8  28.6           61.5   1.0     0.0     1.0   
2   3766.70   49518.024820  175.3  28.9           91.0   1.0     1.0     1.0   
3    980.45  134084.718179  137.3  17.1           32.2   1.0     1.0     1.0   
4   1029.69   49028.625013  162.4  19.7           30.9   1.0     1.0     1.0   

   VitA  
0   0.0  
1   1.0  
2   1.0  
3   1.0  
4   1.0  

[5 rows x 42 columns]

In [44]:
df.columns.tolist()

['SEQN',
 'Age',
 'Household_number',
 'Income ratio',
 'Income',
 'Gender',
 'Race',
 'BirthCountry',
 'EDUC',
 'Subjective health',
 'Health place',
 'Visiting number',
 'Room',
 'Housing type',
 'SpendingFood',
 'Hepatitis A',
 'Hepatitis B',
 'WBC',
 'RBC',
 'PLT',
 'Energy(kcal)',
 'Protein',
 'Carbohydrate',
 'Sugars',
 'Fiber',
 'Fat',
 'Saturated Fatty Acids',
 'monounsaturated fatty acids',
 'Polyunsaturated Fatty Acids',
 'Cholesterol',
 'Caffeine',
 'Theobromine',
 'Alcohol',
 'Moisture',
 'EAT',
 'Height',
 'BMI',
 'Grip Strength',
 'IRON',
 'FOLATE',
 'VitB12',
 'VitA']

In [45]:
df.to_csv('proj_df_Income.csv', index=False)